In [2]:
import pandas as pd

In [5]:
db = pd.read_csv('../data/output.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../data/output.csv'

In [9]:
items = [
    "COD NUMBER",
    "Detail",
    "Detail on NON UIP",
    "Biopsy",
    "Pathology pattern UIP, probable or CHP",
    "Pathology pattern",
    "Extras AP",
    "Diagnosis after Biopsy",
    "Type of telomeric extrapulmonary affectation",
    "Extra",
    "Type of neoplasia",
    "Liver abnormality before diagnosis",
    "Hematological abnormality before diagnosis",
    "Type of liver abnormality",
    "Necessity of transplantation",
    "Cause of death",
    "Identified Infection",
    "Date of death",
    "Pedigree",
    "FamilialvsSporadic",
    "Mutation Type",
    "Severity of telomere shortening - Transform 4",
    "ProgressiveDisease"
]
db2 = db.drop(items, axis=1)

In [4]:
db.insa().sum().sum() - db2.insa().sum().sum()

NameError: name 'db' is not defined

# HASTA AQUI

In [10]:
db.columns

Index(['Pedigree', 'Sex', 'FamilialvsSporadic', 'COD NUMBER',
       'Age at diagnosis', 'Binary diagnosis', 'Final diagnosis', 'TOBACCO',
       'Detail', 'Comorbidities', 'Radiological Pattern', 'Detail on NON UIP',
       'Biopsy', 'Pathology Pattern Binary',
       'Pathology pattern UIP, probable or CHP', 'Pathology pattern',
       'Extras AP', 'Diagnosis after Biopsy', 'Multidsciplinary committee',
       'Pirfenidone', 'Nintedanib', 'Antifibrotic Drug', 'Prednisone',
       'Mycophenolate', 'Treatment', 'Extrapulmonary affectation',
       'Type of telomeric extrapulmonary affectation', 'Extra',
       'Associated lung cancer', 'Other cancer', 'Type of neoplasia',
       'Hematological abnormality before diagnosis',
       'Blood count abnormality at diagnosis', 'Anemia', 'Thrombocytopenia',
       'Thrombocytosis', 'Lymphocytosis', 'Lymphopenia', 'Neutrophilia',
       'Neutropenia', 'Leukocytosis', 'Leukopenia', 'Hematologic Disease',
       'Liver abnormality before diagnosi

In [46]:
print(db['Death'].isnull().sum(), db.shape)
db = db.dropna(subset=['Death'])
print(db['Death'].isnull().sum(), db.shape)


4 (222, 76)
0 (218, 76)


In [47]:
print(db['Progressive disease'].isnull().sum(), db.shape)
db = db.dropna(subset=['Progressive disease'])
print(db['Progressive disease'].isnull().sum(), db.shape)

8 (218, 76)
0 (210, 76)


In [60]:
threshold = len(db) * 0.3
db1 = db.dropna(thresh=threshold, axis=1)

In [61]:
db1.shape
print(db.isna().sum().sum())
print(db1.isna().sum().sum())

2278
654


In [62]:
threshold = len(db.columns) * 0.75
db_cleaned = db1.dropna(thresh=threshold)
print(db_cleaned.shape)
print(db_cleaned.isna().sum().sum())

(209, 67)
640


In [63]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Select only the numerical columns
numerical_cols = db_cleaned.select_dtypes(include=['float64', 'int64']).columns

# Initialize the IterativeImputer
imputer = IterativeImputer(max_iter=10, random_state=0)

# Fit and transform the data
db_cleaned[numerical_cols] = imputer.fit_transform(db_cleaned[numerical_cols])

# Check the result
print(db_cleaned.isna().sum())

Pedigree                                         0
Sex                                              0
FamilialvsSporadic                               0
COD NUMBER                                       0
Age at diagnosis                                 0
                                                ..
More than 1 relative                             0
Genetic mutation studied in patient              0
Severity of telomere shortening                  0
Severity of telomere shortening - Transform 4    0
Progressive disease                              0
Length: 67, dtype: int64


C:\Users\USER\AppData\Local\Temp\ipykernel_10080\3066191376.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db_cleaned[numerical_cols] = imputer.fit_transform(db_cleaned[numerical_cols])


In [68]:
print(db_cleaned['Date of death'].isnull().sum(), db_cleaned.shape)
db_cleaned = db_cleaned.dropna(subset=['Date of death'])
print(db_cleaned['Date of death'].isnull().sum(), db_cleaned.shape)
print(db_cleaned["Transplantation date"].isnull().sum(), db_cleaned.shape)
db_cleaned = db_cleaned.dropna(subset=['Transplantation date'])
print(db_cleaned["Transplantation date"].isnull().sum(), db_cleaned.shape)

0 (198, 67)
0 (198, 67)
4 (198, 67)
0 (194, 67)
ALT                                 0
AST                                 0
ALP                                 0
GGT                                 0
Transaminitis                       0
Cholestasis                         0
Liver disease                       3
FVC (L) at diagnosis                0
FVC (%) at diagnosis                0
DLCO (%) at diagnosis               0
FVC (L) 1 year after diagnosis      0
FVC (%) 1 year after diagnosis      0
DLCO (%) 1 year after diagnosis     0
RadioWorsening2y                    0
Necessity of transplantation        0
Transplantation date                0
Death                               0
Cause of death                     34
Date of death                       0
Type of family history              1
dtype: int64


In [ ]:
# print(db_cleaned.iloc[:,40:60].isna().sum())
print(db_cleaned.isna().sum().sum())
for row in range(db_cleaned.shape[0]):
    l=db_cleaned.iloc[row, :].isna().sum()
    if l>0:
        print(row, l)


In [83]:
# Select only the categorical columns
categorical_cols = db_cleaned.select_dtypes(include=['object']).columns
db_test = db_cleaned.copy()
# Impute missing values in categorical columns with the mode
for col in categorical_cols:
    db_test[col].fillna(db_test[col].mode()[0], inplace=True)

# Check the result
print(db_test.isna().sum().sum())

0


C:\Users\USER\AppData\Local\Temp\ipykernel_10080\1414382188.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  db_test[col].fillna(db_test[col].mode()[0], inplace=True)


In [84]:
db_test.shape

(194, 67)

In [ ]:
from sklearn.preprocessing import train_test_split
